In [78]:
import time
from datetime import datetime, timedelta
import vk_api
import queue
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests

def get_group_ids(spreadsheet_key, sheet_name, range_name):
    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    client = gspread.authorize(credentials)
    spreadsheet = client.open_by_key(spreadsheet_key)
    group_ids_range = spreadsheet.worksheet(sheet_name).range(range_name)
    group_ids = queue.Queue()
    for cell in group_ids_range:
        if cell.value:
            group_ids.put('-' + cell.value)
    return group_ids

def get_or_create_worksheet(spreadsheet, worksheet_name):
    try:
        worksheet = spreadsheet.worksheet(worksheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows="100", cols="30")  # Увеличил количество столбцов
    return worksheet

def clear_worksheet(worksheet):
    worksheet.clear()
    worksheet.update('A1:P1', [['Дата и время проверки', 'Дата публикации', 'Ссылка на пост', 'Текст', 'Комментарии',
                                'Лайки', 'Репосты', 'Просмотры', 'Общий охват', 'Охват подписчиков', 'Вирусный охват',
                                'Охват рекламы', 'Органический охват', 'Рекомендации', 'Клики']])

def collect_stats(group_id, target_spreadsheet_key, num_days, min_post_count, token, telegram_bot_token, telegram_chat_id):
    vk_session = vk_api.VkApi(token=token)
    api = vk_session.get_api()

    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    client = gspread.authorize(credentials)
    target_spreadsheet = client.open_by_key(target_spreadsheet_key)

    worksheet_name = group_id
    worksheet = get_or_create_worksheet(target_spreadsheet, worksheet_name)
    print("Собирается статистика для сообщества:", group_id)

    # Вычисление даты начала периода
    start_date = datetime.now().date() - timedelta(days=num_days)

    posts = api.wall.get(owner_id=group_id, count=200, extended=1)['items']
    if posts:
        sorted_posts = sorted(posts, key=lambda x: x['date'], reverse=False)  # Сортировка по дате
        post_count = 0  # Счетчик собранных постов
        data_rows = []  # Список для хранения строк данных постов
        for post in sorted_posts:
            post_date = datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d %H:%M:%S')

            # Проверка, принадлежит ли пост к периоду
            if datetime.strptime(post_date, '%Y-%m-%d %H:%M:%S').date() >= start_date:
                post_link = f"https://vk.com/wall-{group_id[1:]}_{post['id']}"
                text = post['text']
                comments = post.get('comments', {}).get('count', 0)
                likes = post.get('likes', {}).get('count', 0)
                reposts = post.get('reposts', {}).get('count', 0)
                views = post.get('views', {}).get('count', 0)

                post_stats = api.stats.getPostReach(owner_id=group_id, post_id=post['id'])
                reach_total = post_stats[0].get('reach_total', 0)
                reach_subscribers = post_stats[0].get('reach_subscribers', 0)
                reach_viral = post_stats[0].get('reach_viral', 0)
                reach_ads = post_stats[0].get('reach_ads', 0)
                reach_organic = post_stats[0].get('reach_organic', 0)

                click_stats = api.stats.getPostReach(owner_id=group_id, post_id=post['id'])
                clicks = click_stats[0].get('clicks', 0)

                data_row = [datetime.now().strftime('%Y-%m-%d %H:%M:%S'), post_date, post_link, text, comments, likes,
                            reposts, views, reach_total, reach_subscribers, reach_viral, reach_ads, reach_organic,
                            clicks]

                data_rows.append(data_row)
                post_count += 1

        if post_count >= min_post_count:
            clear_worksheet(worksheet)  # Очистка таблицы перед записью новой статистики
            worksheet.update('A2:P{}'.format(len(data_rows) + 1), data_rows)
            print("Статистика для сообщества", group_id, "собрана и записана в таблицу")
        else:
            print("Недостаточно постов для сообщества", group_id)

def send_telegram_notification(bot_token, chat_id, message):
    requests.get(f"https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={chat_id}&text={message}")

if __name__ == "__main__":
    spreadsheet_key = "1UEmvKWPyJ2nBOt9VdbCAANxEvFLMddu_Uy75N40Ekzw"
    sheet_name = "Фотографы"
    range_name = "ad3:ae8"
    target_spreadsheet_key = "1f97AzhN-YFCofuIeA80ZdDGNRJUajW6o8EBN4XFezGE"
    num_days = 1
    min_post_count = 1

    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    client = gspread.authorize(credentials)
    spreadsheet_key2 = "1f97AzhN-YFCofuIeA80ZdDGNRJUajW6o8EBN4XFezGE"
    worksheet_name = "access"
    target_spreadsheet = client.open_by_key(spreadsheet_key2)
    access_worksheet = get_or_create_worksheet(target_spreadsheet, worksheet_name)
    access_token = access_worksheet.acell('B1').value

    group_ids = get_group_ids(spreadsheet_key, sheet_name, range_name)

    for group_id in list(group_ids.queue):
        collect_stats(group_id, target_spreadsheet_key, num_days, min_post_count, access_token, telegram_bot_token, telegram_chat_id)
        group_ids.get()

    send_telegram_notification(telegram_bot_token, telegram_chat_id, "Сбор статистики завершен")


Собирается статистика для сообщества: -6649905
Статистика для сообщества -6649905 собрана и записана в таблицу
Собирается статистика для сообщества: -18146615
Статистика для сообщества -18146615 собрана и записана в таблицу
Собирается статистика для сообщества: -1623346
Статистика для сообщества -1623346 собрана и записана в таблицу
Собирается статистика для сообщества: -16787932
Статистика для сообщества -16787932 собрана и записана в таблицу
Собирается статистика для сообщества: -25837402
Статистика для сообщества -25837402 собрана и записана в таблицу
Собирается статистика для сообщества: -28825471
Статистика для сообщества -28825471 собрана и записана в таблицу
Собирается статистика для сообщества: -181031
Статистика для сообщества -181031 собрана и записана в таблицу
